In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
#from polyleven import levenshtein
import seaborn as sns
import os
data=pd.read_csv('DATASET/data_with_clusters.csv')
data=data.drop(columns='Unnamed: 0')

max_month=max(data['Month'])
def most_common_seq_for_month(m1):
    most_common_seq1=[]
    for cluster in list(set(m1['Cluster'])):
        if cluster!=0:
            mask=m1['Cluster']==cluster
            temp=m1[mask]
            #hh=Counter(temp['Sequence'])
            a=temp.groupby('Sequence').count().reset_index()
            ms=a[a['EPI ISL']==max(a['EPI ISL'])]['Sequence']
            most_common_seq1.append([cluster,ms.values])
    return most_common_seq1

seq_list=[]
for month in range(3,max_month+1):
    m1=data[data['Month']==month]
    most_common_seq=most_common_seq_for_month(m1)
    seq_list.append(most_common_seq)
edges=[]
for i in range(max_month-3):
    most_common_seq1=seq_list[i]
    most_common_seq2=seq_list[i+1]
    for el1 in most_common_seq1:
        g=el1[1][0]
        for el2 in most_common_seq2:
            j=el2[1][0]
            if g==j:
                #print(f'FRom {el1[0]} to {el2[0]}')
                edges.append((int(el1[0]),int(el2[0])))
                

colors=sns.color_palette("Spectral",max_month)
#colors=sns.color_palette("hls", 13)
#colors=['tab:blue','tab:orange','tab:green','tab:cyan','tab:olive','tab:purple','tab:red','tab:grey','hotpink','chocolate','gold','yellowgreen','tab:pink']
posizioni={}
col=[]
cluster_sequenza=[]
i=0
for lista in seq_list:
    #mese
    j=0
    for seq in lista:
        #cluster
        posizioni[seq[0]]=[i,j]
        j=j+1
        col.append(colors[i])
        #col.append(i)
    i=i+1
        
        
        
        


G=nx.Graph()
max_cluster=int(max(data['Cluster'])+1)
nodi=list(np.arange(1,max_cluster))
G.add_nodes_from(nodi)

G.add_edges_from(edges,color='black',weight=2)
#G.add_edges_from(edges_ch,color='r',weight=2)
fig,ax=plt.subplots(figsize=(14,6))
plt.xlabel('Months')
ed_colors = nx.get_edge_attributes(G,'color').values()
weights = nx.get_edge_attributes(G,'weight').values()
nx.draw(G,pos=posizioni,node_color=col,with_labels=True,edge_color=ed_colors,width=list(weights),node_size=400)

plt.show()

def media_pesata(matrix,weights1,weights2):
    
    matrix=(matrix*np.array(weights1)[:,None])
    matrix=matrix*np.array(weights2)[None,:]
    mean=matrix.sum()/(sum(weights1)*sum(weights2))
    return mean
            
    
def distanza_tra_cluster(cluster1,cluster2):
    #trova cluster1 in mese
    
    
    
   
    try: 
        sequence1=pd.read_csv(f'matrici/{cluster1}_{cluster2}_1.csv')
    
        weights1=sequence1['weights1']

        sequence2=pd.read_csv(f'matrici/{cluster1}_{cluster2}_2.csv')
        
        weights2=sequence2['weights2']
        matrix=np.load(f'matrici new/{cluster1}_{cluster2}.npy')
    except:
        for k,v in diz_cluster.items():
            if cluster1 in v:
                mese=k
        data1=data[data['Month']==mese]
        data2=data[data['Month']==mese+1]
        
        seqs1_t=data1[data1['Cluster']==cluster1]['Sequence']
        seqs2_t=data2[data2['Cluster']==cluster2]['Sequence']
        seqs1=list(set(seqs1_t))
        seqs2=list(set(seqs2_t))
        h1=Counter(seqs1_t)
        weights1=[]
        for ggg in seqs1:
            weights1.append(h1[ggg])

        h2=Counter(seqs2_t)
        weights2=[]
        for ggg in seqs2:
            weights2.append(h2[ggg])



        #print(f'Faccio matrice')
        matrix=np.zeros((len(seqs1),len(seqs2)))
        #print(np.shape(matrix))
        for k,seq1 in enumerate(seqs1):
            for j,seq2 in enumerate(seqs2):
                matrix[k][j]=levenshtein(seq1,seq2)
        print('Fatta matrice')
        np.save(f'matrici/{cluster1}_{cluster2}.npy',matrix)
        sequence1=pd.DataFrame()
        sequence1['seq1']=seqs1
        sequence1['weights1']=weights1
        sequence2=pd.DataFrame()
        sequence2['seq2']=seqs2
        sequence2['weights2']=weights2
        sequence1.to_csv(f'matrici/{cluster1}_{cluster2}_1.csv')
        sequence2.to_csv(f'matrici/{cluster1}_{cluster2}_2.csv')
    dist=media_pesata(matrix,weights1,weights2)
    return dist


!rm -r matrici
os.mkdir('matrici')


#cluster a cui non arriva nulla
edges=np.array(edges)
problematic_clusters=set(range(10,66)).difference(set(np.array(edges[::-1])[:,1]))
cluster_sequenze=[]
for month in range(3,max_month+1):
    tmp=data[data['Month']==month]['Cluster']
    cluster_sequenze.append(list(set(tmp)))
diz_cluster={}
for i,el in enumerate(cluster_sequenze):
    diz_cluster[i+3]=el
print(diz_cluster)



#RETRIVE


weak_edges=[]
for i in np.arange(4,max_month+1):
    clusters=cluster_sequenze[i-3]
    data1=data[data['Month']==i]
    data2=data[data['Month']==i+1]

    for el in clusters:
        if el not in edges[:,0]:
            a=el
            distanze=[]
            bs=[]
            for target in cluster_sequenze[i-2]:
                if target not in edges[:,1]:
                    #print(f'Looking at {el} and {target}')

                    avg=distanza_tra_cluster(el,target)

                    distanze.append(avg)
                    bs.append(target)
            if len(distanze)>0:
                index=distanze.index(min(distanze))
                b=bs[index]
                weak_edges.append([(a,b),min(distanze)]) 

                print(a,b)
                print()

np.save('weak_edges.npy',weak_edges)

#ANALISI LUNGHEZZE
#istogrammi legami

distanza=[]
for legami in edges:
    
        
    print(f' Distanza tra {legami[0]} e {legami[1]}')
    
    dist= distanza_tra_cluster(legami[0],legami[1])
    distanza.append(dist)
    print(dist)
fig,ax=plt.subplots(figsize=(8,5))
plt.hist(distanza)
plt.show()
print()
np.save(distanza,'strong_links.npy')

#istogrammi legami

distanza=[]
for legami in np.array(weak_edges)[:,0]:
    
        
    print(f' Distanza tra {legami[0]} e {legami[1]}')
    
    dist= distanza_tra_cluster(legami[0],legami[1])
    distanza.append(dist)
    print(dist)
fig,ax=plt.subplots(figsize=(8,5))
plt.hist(distanza)
plt.show()
print()
np.save('soft_links.npy',distanza)


#cluster a cui non arriva nulla


cluster_sequenze=[]
for month in range(3,max_month+1):
    tmp=data[data['Month']==month]['Cluster']
    cluster_sequenze.append(list(set(tmp)))
diz_cluster={}
for i,el in enumerate(cluster_sequenze):
    diz_cluster[i+3]=el
diz_cluster
#RETRIVE

weak_edges=np.load('weak_edges.npy',allow_pickle=True)

weak_edges_f=[]
ed=[]
distanze=weak_edges[:,1]
for el in weak_edges[:,0]:
    ed.append((el))
ed=np.array(ed)
problems=[]
filt=[]
for el in set(ed[:,1]):
    if sum(el==ed[:,1])>1:
        problems.append(el)
   
filt=[]
for el in problems:
    indexes=[i for i, e in enumerate(ed) if e[1]==el]
    distanze_tmp=list(distanze[indexes])
    chosen=distanze_tmp.index(min(distanze_tmp))
    filt.append([ed[indexes][chosen],min(distanze_tmp)])
for i,el in enumerate(ed):
    if el[1] not in problems:
        filt.append([el,distanze[i]])
print(filt)
weak_edges=[]
for el in filt:
    if el[1]<20:
        weak_edges.append(el[0])
weak_edges=np.array(weak_edges)

#DISCENZENZA

def find_genitore(STUPIDO):
    dizz={}
    for month in np.arange(4,max_month)[::-1]:

        m1=data[data['Month']==month]
        m2=data[data['Month']==month-1]
        movements=[]

        for cluster_figlio in list(set(m1['Cluster'])):
            
            if cluster_figlio==STUPIDO:
                for cluster_genitore in list(set(m2['Cluster'])):
                    common_seq=list(set(m1[m1['Cluster']==cluster_figlio]['Sequence']).intersection(set(m2[m2['Cluster']==cluster_genitore]['Sequence'])))

                    for seq in common_seq:
                        seq_dopo=sum(m1[m1['Cluster']==cluster_figlio]['Sequence']==seq)
                        seq_prima=sum(m2[m2['Cluster']==cluster_genitore]['Sequence']==seq)

                        if seq_dopo>seq_prima:
                            if (cluster_figlio,cluster_genitore) in dizz:
                                dizz[(cluster_figlio,cluster_genitore)]=dizz[(cluster_figlio,cluster_genitore)]+seq_dopo
                            else:
                                dizz[(cluster_figlio,cluster_genitore)]=seq_dopo
    
    if len(dizz)>0:
        obiettivo=max(list(dizz.values()))
        for k,v in dizz.items():
            if v==obiettivo:
                return (k[1],k[0])

def retrive_chains(tot_edges):    
    cose=np.array(tot_edges).astype(int)
    cose=sorted(cose, key=lambda x: x[0])
    cose=np.array(cose)
    chains=[]
    chain=[]

    prec=cose[0]

    mask=[True if (x!=prec).all() else False for x in cose] 
    cose=cose[mask]
    chain.append(prec[0])
    chain.append(prec[1])
    #print(cose)
    while len(cose)>0:
        
        nella_catena=False
        for el in cose:
            
            if (el[0]==prec[1]).all():
                chain.append(el[1])
                prec=el
                nella_catena=True

                mask=[True if (x!=el).all() else False for x in cose] 
                cose=cose[mask]

        if nella_catena==False or len(cose)==0:
            chains.append(chain)
            #print(chain)
            if len(cose)>0:
                chain=[]

                prec=cose[0]
                #print(prec)
                cose=cose[1:]
                chain.append(prec[0])
                chain.append(prec[1])
    #print(chains)
    return chains

weak_edges=np.array(weak_edges)
filtered_weak_edges=list(map(lambda x: (x[0],x[1]),weak_edges))
tot_edges=filtered_weak_edges+list(edges)
tot_edges.sort(key=lambda x:x[0])
chains=retrive_chains(tot_edges)
chains


weak_edges=np.array(weak_edges)
filtered_weak_edges=list(map(lambda x: (x[0],x[1]),weak_edges))
tot_edges=filtered_weak_edges+list(edges)
tot_edges.sort(key=lambda x:x[0])


desc_edges=[]
for chain in chains:
    
    cluster_figlio=chain[0]
    
    for k,v in diz_cluster.items():
        if cluster_figlio in v:
            month=k
    if k!=3:
        b=find_genitore(cluster_figlio)
        if b!=None:
            desc_edges.append(b)
desc_edges.append(find_genitore(37))
desc_edges

G=nx.Graph()

nodi=list(np.arange(1,max_cluster))
G.add_nodes_from(nodi)

G.add_edges_from(desc_edges,color='y',weight=1)
G.add_edges_from(weak_edges,color='r',weight=2)
G.add_edges_from(edges,color='black',weight=2)
fig,ax=plt.subplots(figsize=(14,6))
plt.xlabel('Months')
ed_colors = nx.get_edge_attributes(G,'color').values()
weights = nx.get_edge_attributes(G,'weight').values()
nx.draw(G,pos=posizioni,node_color=col,with_labels=True,edge_color=ed_colors,width=list(weights),node_size=400)

plt.show()


giax=[]
giap=[]
#colors=sns.color_palette("Spectral",max_month)
colors=['tab:blue','tab:red','hotpink','gold','tab:green','gold','tab:red','tab:pink','tab:grey','hotpink','gold','navy','lightgreen']
name=['B.1.1.1','B1.117','B.1.117+L18F','B.1.1.7','B.1.617.2',0,0,0,0]
fig,ax=plt.subplots(figsize=(10,6))
percs=[]
for h,chain in enumerate(chains):
    if len(chain)>1:
        perc=[]
        for i,clu_in_month in enumerate(cluster_sequenze):

            data1=data[data['Month']==i+3]
            den=len(data1)

            check=any(item in clu_in_month for item in chain)
            if check:
                for el in clu_in_month:
                    if el in chain:
                        nom=sum(data1['Cluster']==el)
                        perc.append(nom/den)
                        if el==chain[0]:
                            start=i+3
                        if el==chain[-1]:
                            fine=i+3


        k=1
        if chain[-1] not in cluster_sequenze[-1]:
            perc.insert(len(perc),0)
            k=0
        j=0
        if chain[0] not in cluster_sequenze[0]:
            perc.insert(0,0)
            j=1
        plt.plot(range(start-j,fine+2-k),perc,label=name[h],c=colors[h])
        #plt.scatter(range(start-j,fine+2-k),perc,c=colors[h])
        giax.append(range(start-j,fine+2-k))
        giap.append(perc)
        percs.append([perc,h])

plt.legend()
plt.xlabel('Months')
plt.ylabel('Percentage')
plt.title('Montly percentage per chain')
#plt.savefig('images/perc.png')
plt.show()      

nodi=list(np.arange(1,max_cluster))
print(diz_cluster)
diz={}
for k,v in diz_cluster.items():
    for el in v:
        diz[el]=k
print(diz)
posizioni={}

chains=sorted(chains,key=lambda x: len(x),reverse=True)
for j,chain in enumerate(chains):
    for i,cluster in enumerate(chain):
        start=diz[chain[0]]-3
        #print(start)
        posizioni[int(cluster)]=(start+i,j)
print(posizioni)
for cluster in range(1,len(nodi)+1):
    if cluster not in posizioni: #cluster avanzato
        
        ascissa=diz[cluster]-3
        ordinate=[]
        for cluster_nel_mese in diz_cluster[ascissa+3]:
            
            if cluster_nel_mese in posizioni:
                
                #print(cluster_nel_mese)
                ordinate.append(posizioni[cluster_nel_mese][1])
       
        ordinata=max(ordinate)+1
        posizioni[cluster]=(ascissa,ordinata)
        
for el in chains[2]:
     posizioni[el]=(posizioni[el][0],posizioni[el][1]+1)
        
for el in chains[3]:
    posizioni[el]=(posizioni[el][0],posizioni[el][1]-1)
for el in [15,22,16,21]:
    posizioni[el]=(posizioni[el][0],posizioni[el][1]-1)
            
posizioni[21]=(posizioni[21][0],posizioni[21][1])
posizioni[37]=(posizioni[37][0],posizioni[37][1]-1)
posizioni[40]=(posizioni[40][0],posizioni[40][1]-1)
colors=sns.color_palette("Spectral",19)[1:-1]
col=[]
cluster_sequenza=[]
i=0
for lista in seq_list:
    #mese
    j=0
    for seq in lista:
        #cluster
        j=j+1
        col.append(colors[i])
        #col.append(i)
    i=i+1

G=nx.Graph()
max_cluster=int(max_cluster)

G.add_nodes_from(nodi)
G.add_edges_from(desc_edges,color='grey',weight=1)
G.add_edges_from(weak_edges,color='r',weight=2)
G.add_edges_from(edges,color='black',weight=2)
fig,ax=plt.subplots(figsize=(10,5))
plt.xlabel('Months')
ed_colors = nx.get_edge_attributes(G,'color').values()
weights = nx.get_edge_attributes(G,'weight').values()
nx.draw(G,pos=posizioni,node_color=col,with_labels=True,edge_color=ed_colors,width=list(weights),node_size=400)
plt.savefig('images/GRAPH_COMPLETE.png')
plt.show()

#distanza tra cluster successivi nella catena

for chain in chains:
    distanza=[]
    x_axis=[]
    if len(chain)>2:
        for i in range(len(chain)-1):
            print(f' Distanza tra {chain[i]} e {chain[i+1]}')
            
            dist= distanza_tra_cluster(chain[i],chain[i+1])
            distanza.append(dist)
            x_axis.append(f'{int(chain[i])}-{int(chain[i+1])}')
            #print(dist)
        fig,ax=plt.subplots(figsize=(8,5))
        plt.plot(x_axis,distanza)
        plt.title(chain)
        plt.savefig(f'dist cluster/{chain[0]}')
        plt.show()
        print()
        
counts=pd.read_csv('DATASET/data_2021-Jul-07.csv')
months=[]
xs=[]
name=['B.1.1.1','B.1.117+L18F','B1.117','B.1.1.7',0,0,'B.1.617.2',0,0,0]
for x in counts['date']:
    year,month,_=x.split('-')
    if year=='2021':
        
        month=int(month)+12
    months.append(int(month))
counts['Month']=months
counts=list(counts.groupby('Month').sum().reset_index()['newCasesBySpecimenDate'][2:])

diz_cluster
diz_cluster2={}
for k,v in diz_cluster.items():
    for el in v:
        diz_cluster2[el]=k
diz_cluster2
print(counts)
        
colors=['tab:blue','hotpink','gold','tab:red','tab:olive','tab:green','tab:green','tab:pink','gold','hotpink','gold','navy','lightgreen']
fig,ax=plt.subplots(figsize=(10,6))
for h,chain in enumerate(chains):
    if len(chain)>1 and chain[0]!=5 and chain[0]!=15:
        perc=[]
        for i,clu_in_month in enumerate(cluster_sequenze):

            dataframe=data[data['Month']==i+3]
            den=len(dataframe)

            check=any(item in clu_in_month for item in chain)
            if check:
                for el in clu_in_month:
                    if el in chain:
                        s_month=diz_cluster2[el]
                        nom=sum(dataframe['Cluster']==el)
                        perc.append((nom/den)*counts[s_month-3])
                        if el==chain[0]:
                            start=i+3
                        if el==chain[-1]:
                            fine=i+3


        k=1
        if chain[-1] not in cluster_sequenze[-1]:
            perc.insert(len(perc),0)
            k=0
        j=0
        if chain[0] not in cluster_sequenze[0]:
            perc.insert(0,0)
            j=1
        plt.plot(range(start-j,fine+2-k),perc,label=name[h],c=colors[h])
        xs.append(range(start-j,fine+2-k))
plt.legend()
plt.xlabel('Months')
plt.ylabel('Number of new infected')
plt.title('Total infected')
#plt.yscale('log')
plt.savefig('images/percentages_tot.png')
plt.show()      